In [25]:
# for string distance algorithm
import jellyfish
import my_load
import pandas as pd
import dask.dataframe as dd
import gc

In [24]:
# # select desired columns tab10, 
# med_procedure = tab10[["arb_person_id", "arb_encounter_id", "ProcedureName"]]
# # select desired columns tab11, Lab
# # lab_procedure = tab11[["arb_person_id", "arb_encounter_id"]]

In [7]:
# # Inner join MemoryError 
# # MemoryError: Unable to allocate 29.7 GiB for an array with shape (3983626340,) and data type int64
# innerJoin_personId_encoun_id = pd.merge(med_procedure, tab11, on=["arb_person_id", "arb_encounter_id"], how="inner")

In [8]:
# path and cols to read in
file_path_tab11 = ("//data/dept/SOM/ACCORDS/PiFolders/PI_Colborn/UCHealthSOR/Data/C2730_20240628"
                       "/C2730_Table11_Lab_20240223.csv")
# dtype = {"EpicCptCode": "object", "LabResult": "object"}
cols11 = ["arb_person_id", "arb_encounter_id", 'LabComponentName', 'LabPanelName']

file_path_tab10 = ("//data/dept/SOM/ACCORDS/PiFolders/PI_Colborn/UCHealthSOR/Data/C2730_20240628"
                       "/C2730_Table10_CUMedicineProcedure_20240223.csv")
cols10 = ["arb_person_id", "arb_encounter_id", "ProcedureCategoryName", "ProcedureName"]

In [9]:
# read in data
tab10 = my_load.load_csv_dask_cols(file_path_tab10, cols=cols10)
print(tab10.shape)
print(tab10.memory_usage(index=True).sum()/1000000000)
tab11 = my_load.load_csv_dask_cols(file_path_tab11, cols=cols11)
print(tab11.shape)
print(tab11.memory_usage(index=True).sum()/1000000000)

(7611644, 4)
0.30446576
(187984110, 4)
7.5193644


In [10]:
# drop duplicates 
tab10.drop_duplicates(inplace=True)
tab11.drop_duplicates(inplace=True)
print(tab10.shape)
print(tab11.shape)

(6110898, 4)
(114123508, 4)


In [11]:
# try merge deduplicated tables using dask
join10_11 = dd.merge(tab10, tab11, on=["arb_person_id", "arb_encounter_id"], how="inner")
del tab11
del tab10

In [13]:
# manual garbage collection
gc.collect()

2736

In [14]:
# transfer object from disk to memory
result = join10_11.compute()

In [16]:
del join10_11

In [17]:
# manual garbage collection
gc.collect()

28

In [18]:
print(result.shape)

(263895898, 6)


In [19]:
result.head(20)

,arb_person_id,arb_encounter_id,ProcedureCategoryName,ProcedureName,LabComponentName,LabPanelName
0,7582269443,65454259158,EVAL AND MGMT,EMERGENCY DEPT VISIT HI MDM,Hematocrit,CBC NO AUTO DIFF
1,7582269443,65454259158,EVAL AND MGMT,EMERGENCY DEPT VISIT HI MDM,Shigella/Enteroinvasive E coli,GI PCR PANEL (AFFILIATES ONLY)
2,7582269443,65454259158,EVAL AND MGMT,EMERGENCY DEPT VISIT HI MDM,Chloride Serum/Plasma,BASIC METABOLIC PANEL
3,7582269443,65454259158,EVAL AND MGMT,EMERGENCY DEPT VISIT HI MDM,Product Blood Type,PREPARE RBCS FOR TRANSFUSION
4,7582269443,65454259158,EVAL AND MGMT,EMERGENCY DEPT VISIT HI MDM,White Blood Cells Urine,UA DIPSTICK W/ REFLEX TO MICROSCOPIC EXAM IF I...
5,7582269443,65454259158,EVAL AND MGMT,EMERGENCY DEPT VISIT HI MDM,Body Fluid Lymphocytes,"DIFFERENTIAL, BODY FLUID (PERFORMABLE)"
6,7582269443,65454259158,EVAL AND MGMT,EMERGENCY DEPT VISIT HI MDM,Protein Total Serum/Plasma,HEPATIC FUNCTION PANEL
7,7582269443,65454259158,EVAL AND MGMT,EMERGENCY DEPT VISIT HI MDM,Urine Culture,URINE CULTURE
8,7582269443,65454259158,EVAL AND MGMT,EMERGENCY DEPT VISIT HI MDM,Parainfluenza 4,RESPIRATORY VIRAL PANEL PCR
9,7582269443,65454259158,EVAL AND MGMT,EMERGENCY DEPT VISIT HI MDM,Blood Urea Nitrogen,BASIC METABOLIC PANEL


In [20]:
# # test join with 1/10 data
# tab10 = my_load.load_csv_dask_cols(file_path_tab10, cols=cols10)
# print(tab10.shape)
# print(tab10.memory_usage(index=True).sum()/1000000000)
# tab11 = my_load.load_csv_dask_cols(file_path_tab11, cols=cols11)
# print(tab11.shape)
# print(tab11.memory_usage(index=True).sum()/1000000000)
# t10 = tab10.sample(frac=0.1, replace=False, random_state=1)
# t11 = tab11.sample(frac=0.1, replace=False, random_state=1)

# join10_11 = dd.merge(t10, t11, on=["arb_person_id", "arb_encounter_id"], how="inner")
# del tab11
# del tab10
# gc.collect()
# test_join = join10_11.compute()
# test_join.head(200)

In [ ]:
# using jellyfish package to calculate string distances